# Big data analysis with Dask

In [ ]:
import json
import gcsfs

In [ ]:
token = json.load(open("prep/credentials.json"))
fs = gcsfs.GCSFileSystem(token=token)
storage_options={"token": token}

In [ ]:
import dask_gateway

gateway = dask_gateway.Gateway()

In [ ]:
options = gateway.cluster_options(use_local_defaults=False)
options

In [ ]:
cluster = gateway.new_cluster(options)
cluster

In [ ]:
client = cluster.get_client()
client

## Partitions

* map_partitions


## Data locality

## Task graph

## Persist



## Meta

## Diagnostics with dashboard

## Shuffling

- set-index